In [ ]:
import h3
from h3_transformation import H3Transformation

## Introduction to H3

### H3Index - The core of H3

The H3 system assigns a unique hierarchical index (H3Index) to each cell (hexagon/pentagon).
- Each directed edge (Mode 2) and vertex (Mode 4) is assigned an index based on its origin or owner cell, respectively. 



### H3Index Representation

An H3Index is the 64-bit `integer` representation of an H3 index, which may be one of multiple modes to indicate the concept being indexed.
- Mode 0 is reserved and indicates an **invalid H3 index**.
- Mode 1 is an **H3 Cell** (Hexagon/Pentagon) index.
- Mode 2 is an **H3 Directed Edge** (Cell A -> Cell B) index.
- Mode 3 is planned to be a **bidirectional edge** (Cell A <-> Cell B).
- Mode 4 is an **H3 Vertex** (i.e. a single vertex of an H3 Cell).

### Key points

- In H3 version 3.x, whenever the term **h3** is used, it refers to the `H3Index` in *Mode 1* of a cell.
- By default, the H3Index is of type `string`, which represent the equivalent hexadecimal of the 64-bit `integer` using lowercase letters.

---

## Main concepts

We will go through the fundamental API usages of H3, please note that all the examples in this notebook are from H3 version 3.x (https://h3geo.org/docs/3.x/).

A lot of the methods will look very different in H3 version 4.x (https://h3geo.org/docs/), which is currently in beta.

### Indexing functions

These function are used for finding the H3 index containing coordinates, and for finding the center and boundary of H3 indexes.

#### [geoToH3](https://h3geo.org/docs/3.x/api/indexing#geotoh3)

Indexes the location at the specified resolution, returning the index of the cell containing the location. This buckets the geographic point into the H3 grid. See the algorithm description for more information.

Returns 0 on error.

In [ ]:
latitude = 10.7780202
longitude = 106.7019901
resolution = 12  # For geodesic statistics of resolution, visit: https://h3geo.org/docs/3.x/core-library/restable

# Generate an H3 cell, which is an H3Index (string) in Mode 1. We will continue to use this H3 Cell thoughout the notebook.
h3_cell = h3.geo_to_h3(lat=latitude, lng=longitude, resolution=resolution)
h3_cell

#### [h3ToGeo](https://h3geo.org/docs/3.x/api/indexing#h3togeo)

Finds the center (lat, lng) of the cell in grid space. See the algorithm description for more information.

The center will drift versus the centroid of the cell on Earth due to distortion from the gnomonic projection within the icosahedron face it resides on and its distance from the center of the icosahedron face.



In [ ]:
# Returns the (lat, lng) of the center of the H3 cell
h3.h3_to_geo(h3_cell) # yes, h3_to_geo sounds pretty confusing, and H3 change this method to cell_to_latlng in v4.x

#### [h3ToGeoBoundary](https://h3geo.org/docs/3.x/api/indexing#h3togeoboundary)

Finds the boundary (vertices of the hexagon/pentagon) of the cell.

To learn more about the implementation, take a look at the documentation: [Generate the cell boundary in latitude/longitude coordinates of an H3Index cell](https://h3geo.org/docs/3.x/core-library/h3ToGeoBoundaryDesc/).

In [ ]:
boundary = h3.h3_to_geo_boundary(h3_cell, geo_json=False)
boundary_geojson = h3.h3_to_geo_boundary(
    h3_cell,
    geo_json=True,  # returns output in GeoJSON format: [lng, lat] order; first and last coordinates are identical.
)

print(boundary, boundary_geojson, sep="\n")

### Inspection functions

These functions provide metadata about an H3 index, such as its resolution or base cell, and provide utilities for converting into and out of the 64-bit representation of an H3 index.

#### [h3GetResolution](https://h3geo.org/docs/3.x/api/inspection#h3getresolution)

Returns the resolution of the index.

In [ ]:
h3.h3_get_resolution(h3_cell)

#### [h3GetBaseCell](https://h3geo.org/docs/3.x/api/inspection#h3getbasecell)

Returns the base cell number of the index.

In [ ]:
h3.h3_get_base_cell(h3_cell)

#### [stringToH3](https://h3geo.org/docs/3.x/api/inspection#stringtoh3)

Converts the string representation to `H3Index` (uint64_t) representation. 
- By default, all the H3 indexes are represented as a `string`. Whenever the term `H3Index` is mentioned, it refers to the `uint64_t` representation of the index.

Returns 0 on error.

In [ ]:
h3_cell_uint64_t = h3.string_to_h3(h3_cell)

print(type(h3_cell_uint64_t), h3_cell_uint64_t)

#### [h3ToString](https://h3geo.org/docs/3.x/api/inspection#h3tostring)

Converts the `H3Index` (uint64_t) representation of the index to the `string` representation. str must be at least of length 17.

In [ ]:
h3_cell_str = h3.h3_to_string(h3_cell_uint64_t)

print("h3_cell_str       :", type(h3_cell_str), h3_cell_str)    # `string` representation of the H3 Cell.
print("h3_cell (default) :", type(h3_cell), h3_cell)            # By default, any H3 Cell generated is already in the `string` representation.

#### [h3IsValid](https://h3geo.org/docs/3.x/api/inspection#h3isvalid)

Returns non-zero if this is a valid H3 index.

In [ ]:
h3_cell_check           = h3.h3_is_valid(h3_cell)
h3_cell_str_check       = h3.h3_is_valid(h3_cell_str)
h3_cell_uint64_t_check  = h3.h3_is_valid(h3_cell_uint64_t)

# However, it seems that `h3_is_valid` only works with `string` representation of the H3 Cell. I guess in v3.x, the term `h3` always refers to the `string` representation of the H3 Cell :-). 
h3_cell_check, h3_cell_str_check, h3_cell_uint64_t_check

## Children

In [ ]:
# children of a H3 cell
children = h3.h3_to_children(h3_cell, resolution + 1)
print(children)

## Traversal

Grid traversal allows finding cells in the vicinity of an origin cell, and determining how to traverse the grid from one cell to another.

In [ ]:
# Set distance for all traversal examples
distance_km = 5

### K-rings

k-rings produces indices within k distance of the origin index.

k-ring 0 is defined as the origin index, k-ring 1 is defined as k-ring 0 and all neighboring indices, and so on.

In [ ]:
# K-ring of a H3 cell:
k_ring_5km = h3.k_ring(h3_cell, distance_km)
print("k_ring_5km type:", type(k_ring_5km))

for cell in k_ring_5km:
    print(f"Cell: {cell}, Center: {h3.h3_to_geo(cell)}")

### Hex Range Distance

hexRange produces indexes within k distance of the origin index. Output behavior is undefined when one of the indexes returned by this function is a pentagon or is in the pentagon distortion area.

k-ring 0 is defined as the origin index, k-ring 1 is defined as k-ring 0 and all neighboring indexes, and so on.

In [ ]:
# hex_range_distance
rings_5km = h3.hex_range_distances(h3_cell, distance_km)

current_ring = 0
for ring in rings_5km:
    current_ring += 1
    print(f"Ring {current_ring} has {len(ring)} cells")

    for cell in ring:
        print(f"Cell: {cell}, Center: {h3.h3_to_geo(cell)}")

    print("-------------------")

## H3 to GeoJSON

In [ ]:
geojson = H3Transformation.cells_to_geojson(k_ring_5km)
print(geojson)